In [1]:
from sqlalchemy import create_engine
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from env_vars import *
import pandas as pd
import sqlite3
from sqlalchemy import create_engine

# Connection to DB for executions

In [2]:
engine = create_engine('sqlite:///song_list_v2.db', echo=False)

# Functions for use in this notebook

In [3]:
def authorize():
    client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID,client_secret=CLIENT_SECRET)
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [4]:
client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID,client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [5]:
def get_features(trackID):
        return sp.audio_features(trackID)[0]
    
get_features('54flyrjcdnQdco7300avMJ')

{'danceability': 0.693,
 'energy': 0.497,
 'key': 2,
 'loudness': -7.316,
 'mode': 1,
 'speechiness': 0.119,
 'acousticness': 0.679,
 'instrumentalness': 0,
 'liveness': 0.258,
 'valence': 0.473,
 'tempo': 81.308,
 'type': 'audio_features',
 'id': '54flyrjcdnQdco7300avMJ',
 'uri': 'spotify:track:54flyrjcdnQdco7300avMJ',
 'track_href': 'https://api.spotify.com/v1/tracks/54flyrjcdnQdco7300avMJ',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/54flyrjcdnQdco7300avMJ',
 'duration_ms': 122067,
 'time_signature': 4}

In [6]:
def get_artist_id(trackID):
    return sp.track(trackID)['artists'][0]['id']
    


get_artist_id('54flyrjcdnQdco7300avMJ')

'1dfeR4HaWDbWqFHLkxsg1d'

In [7]:
def get_genres(artistID):
    return sp.artist(artistID)['genres']


get_genres('1dfeR4HaWDbWqFHLkxsg1d')

['glam rock', 'rock']

# [COMPLETED] Inserts Genre Value into Genres Column

In [ ]:
for item in engine.execute("SELECT songid FROM features").fetchall()[515122:]:    
    songid = item.values()[0]
    artistid = get_artist_id(songid)
    genres = get_genres(artistid)
    try:
        engine.execute(f'UPDATE features SET genres = \"{genres}\" WHERE songid = \"{songid}\"')
        authorize()
    except:
        print("couldn't insert")
        continue

In [13]:
# null = []
# for genre in genres_list:
#     if genre == []:
#         null.append(genre)

# Convert to PD DF - For Exploration - SEE NULL VALUE RESULT

In [64]:
con = sqlite3.connect("./song_list_v2.db")
df = pd.read_sql_query("SELECT * from features", con)

In [65]:
df.isnull().sum()

index               0
acousticness        0
artist              0
danceability        0
duration_ms         0
energy              0
instrumentalness    0
key                 0
liveness            0
loudness            0
mode                0
songid              0
speechiness         0
tempo               0
time_signature      0
track               1
valence             0
genres              0
dtype: int64

# [Complete] - Create List of all Null Values for Investigation

In [69]:
nan_rows = df[df['genres'].isnull()].index.tolist()

In [70]:
len(nan_rows)

0

In [71]:
# df.loc[nan_rows[0]]

In [72]:
df.loc[2:5]

,index,acousticness,artist,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,songid,speechiness,tempo,time_signature,track,valence,genres
2,2,0.000725,Der Mystic,0.430,514290.0,0.960,0.925000,5.0,0.123,-7.741,0.0,2poHURuOfVNbzZdivAwtOH,0.0431,140.001,4.0,Tangle Of Aspens,0.225,[]
3,3,0.245000,Sasha / Turbulence,0.663,212413.0,0.677,0.000000,3.0,0.350,-4.117,0.0,1jg9hZnReygpBvV2axGuPy,0.0755,152.085,4.0,We Have Got Love,0.879,[]
4,4,0.002760,Kris Kross,0.859,221200.0,0.741,0.000000,11.0,0.325,-12.329,0.0,3GsS8jzoixpCnp4jDWCEvb,0.2710,98.082,4.0,2 Da Beat Ch'yall,0.529,"['hip hop', 'new jack swing', 'rap']"
5,5,0.348000,Jorge Negrete,0.361,173573.0,0.483,0.000003,7.0,0.177,-6.875,1.0,0kq75szR7uDEYrZkT2c4Ry,0.0287,94.538,4.0,El hijo del pueblo,0.682,"['bolero', 'mariachi', 'ranchera']"


# Remove track NULL value and reset index and pickle new DF

In [79]:
df = df.reset_index()

In [80]:
df = df.drop(columns="index")

In [81]:
df.loc[[117524], ['track']] = "Mashin For Mine / Summer In The City"

In [83]:
df.genres.valye_

,level_0,acousticness,artist,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,songid,speechiness,tempo,time_signature,track,valence,genres
728151,728151,0.8200,Instupendo,0.418,235771.0,0.204,0.936,8.0,0.0866,-13.522,1.0,1xm8J6EFMA6N8JDqH8vzuz,0.0370,110.022,4.0,Boy,0.0375,['lo-fi beats']
728152,728152,0.0704,Mitchell Tenpenny,0.614,209032.0,0.952,0.000,6.0,0.3620,-2.782,1.0,4NwmHBjPb9i9N3naLCMVCG,0.0718,148.079,4.0,Truck I Drove in High School,0.8200,"['contemporary country', 'country pop', 'heart..."
728153,728153,0.0640,Cold War Kids,0.343,324920.0,0.487,0.000,2.0,0.1070,-9.462,1.0,5Xo8AsEz0gpW6Rpo2jXvBN,0.0473,171.217,4.0,Goodnight Tennessee,0.3650,"['garage rock', 'indie pop', 'indie rock', 'mo..."
728154,728154,0.0292,Elevation Worship,0.380,423377.0,0.399,0.000,2.0,0.8910,-8.466,1.0,4Fnz6vDqufd3ens0Gf9LC5,0.0321,125.800,3.0,Hallelujah Here Below,0.1050,"['ccm', 'christian music', 'world worship', 'w..."
728155,728155,0.1010,Jae Mansa,0.834,240353.0,0.517,0.000,5.0,0.2720,-5.336,1.0,2tolmRzbUfgL5KRplIqHlu,0.1490,129.932,4.0,I Swear,0.1930,[]


# Print Values for Test - Complete

In [73]:
def print_values(songid):    
    artistid = get_artist_id(songid)
    print(artistid)
    genres = get_genres(artistid)
    print(genres)

print_values('4LisLrspORq8IagdLHWkf7')

4zMssRZ6iG7sJm54RwFozt
['cartoon', "children's music"]


# [Completed] 
# Retry Genre Lookup / SQL Update on Collected Db for Null Rows

In [75]:
nan_rows

[]

In [77]:
# for row_index in nan_rows:
#     songid = df.loc[row_index]["songid"]
#     artistid = get_artist_id(songid)
#     genres = get_genres(artistid)
#     newgenres = '['
#     for item in g:
#         newgenres += "'" + item + "',"
#     newgenres = words[:-1]
#     newgenres += "]" 
#     try:
#         engine.execute(f'UPDATE features SET genres = \"{newgenres}\" WHERE songid = \"{songid}\"')
#         authorize()
#     except:
#         print(f"couldn't insert {songid} into table")
#         continue
        